In [ ]:
import requests
from bs4 import BeautifulSoup

def get_all_links():
    # Use a session to preserve cookies
    session = requests.Session()
    
    # Step 1: Visit homepage to get cookies
    main_url = "https://keralapolice.gov.in"
    session.get(main_url, verify=False)  # verify=False to ignore SSL cert issues (use cautiously)
    print("✅ Fetched homepage (cookies set)")

    # Step 2: Now request the English version in the same session
    en_url = "https://keralapolice.gov.in/switch/language/en"
    response = session.get(en_url, verify=False)
    print("✅ Fetched English page")

    # Optional: Check if it's really in English
    if "About Kerala Police" in response.text:
        print("✅ Confirmed: Page is in English")
    else:
        print("⚠️ Warning: May still be in Malayalam")

    # Parse with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    linkaddrs = [link.get('href') for link in links if link.get('href')]
    return linkaddrs


# Run it
linkaddrs = get_all_links()
print("\n🔗 Extracted Links:")
for link in linkaddrs:
    print(link)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
from openai import OpenAI
import os
import dotenv

# Load environment
dotenv.load_dotenv()
NVIDIA_API_KEY = os.getenv("nvidia_api_key")

if not NVIDIA_API_KEY:
    raise ValueError("NVIDIA_API_KEY not found in .env")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY,
)
base_url = "https://keralapolice.gov.in"
session = requests.Session()

def extract_main_content(page_path: str):
   
    try:
        page_path=page_path.strip()
        session.get(f"{base_url}", timeout=10, verify=False)
        print("Initial page loaded")
        session.get(f"{base_url}/switch/language/en",verify=False)
        print("loaded english content")
        full_url = f"{base_url}/{page_path}" if page_path else base_url
        response = session.get(full_url, timeout=10, verify=False)
        soup = BeautifulSoup(response.text, 'html.parser')

        for tag in soup(["script", "style", "nav", "footer", "header", "aside"]):
            tag.decompose()

        title = soup.title.string.strip() if soup.title else "Kerala Police"

        main_content = soup.find("body")

        if main_content:
            lines = (line.strip() for line in main_content.stripped_strings)
            body_text = "\n".join(line for line in lines if line)
        else:
            print("No content found")
            body_text = ""
            return 0

        return {
            "url": full_url,
            "title": title,
            "content": body_text
        }

    except Exception as e:
        print(f"Error scraping {full_url}: {e}")
        return {"url": full_url, "title": "", "content": ""}


data = extract_main_content("page/vision-mission")

prompt = f"""
You are an expert data curator for a making RAG (Retrieval-Augmented Generation) document retrieval system.
Your task is to extract structured, useful information from the Kerala Police website.
You will be given the content of a webpage, and your task is to extract and return the following:
    -extract all the content that will be relevant for building and provide accordingly in the output.
    -if found like different topics, then give all the diffrent topics as differnt title and content.
    -extract the information, and give the content intelligently, you can modify the content you extracted according to my use case.
    -I will be extracting the title you provided as the documnet title for making rag document embeddi.
    -I will be extracting the content you provided as the documnet content for making rag document embeddi.
Given the following webpage content:

TITLE: {data['title']}
CONTENT:
{data['content']}

Extract the contents in **valid JSON format only** (no markdown, no explanation). If found or have multiple contents, you can output the each content in this format:

{{
  "Title": "Content you extracted",
  "Content": "Content you extracted"
  
}}

Important:
- Only include links that are internal (start with / or https://keralapolice.gov.in)
- Do not invent information.
- If a field is unknown, use empty string or empty list.
- Output ONLY the JSON object.
"""
if data:
    try:
        completion = client.chat.completions.create(
            model="qwen/qwen3-coder-480b-a35b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            top_p=0.7,
            max_tokens=1024,
        )
        result = completion.choices[0].message.content.strip()

        # Try to parse JSON (in case LLM adds ```json...```)
        if result.startswith("```json"):
            result = result[7:-3].strip()
        elif result.startswith("```"):
            result = result[3:-3].strip()

        structured_data = json.loads(result)
        print(json.dumps(structured_data, indent=2, ensure_ascii=False))
        filename = "vision and mission.txt"
        with open(filename, "w") as f:
            f.write(json.dumps(structured_data, indent=2, ensure_ascii=False))

    except Exception as e:
        print(f"❌ Error: {e}")
        print("Raw LLM output:")
        print(result)


In [ ]:
#rag dataset prepare to csv
import pandas as pd
import json
import os

folder="data"

#open all the files in the folder
files = [f for f in os.listdir(folder) if f.endswith(".txt")]

print("Found", len(files), "files in", folder)

In [ ]:
from openai import OpenAI
import os
import json
NVIDIA_API_KEY = os.getenv("nvidia_api_key")

if not NVIDIA_API_KEY:
    raise ValueError("NVIDIA_API_KEY not found in .env")

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY,
)
def chunksplit(data):
    prompt=f"""
    You are an expert data curator for a making RAG (Retrieval-Augmented Generation) document retrieval system.
    You will be given a dictionary with the following keys:
        Title: The title of the document.
        Content: The content of the document.
        
    the content of the documnet is greater than 250 words.
    your job is to return the content in chunks of 250 words in the following format of multiple dictionary in a dictionary/json as in the format without losing the
    meaning of the content, that is dont trim the content upto just 249 words, the content ust have a completeness. dont just trim acoording to the size of words, output the data 
    in the following, dont include any things, return the list only:
    {{
        {{
            "Title": "The title of the data",
            "Content": Trimmed content 1
        }},
        {{
            "Title": "The title of the data",
            "Content": Trimmed content 2
        }},
        etc
    }}
    You must not modify or remove it. Make it efficeint trimming for the RAG.

    The dictionary data:
    {data}"""
    
    try:
        print("prompt given:\n",prompt)
        completion= client.chat.completions.create(
            model="openai/gpt-oss-120b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            top_p=0.2,
            stream=False
        )
        reasoning = getattr(completion.choices[0].message, "reasoning_content", None)
        if reasoning:
            print(reasoning)
        print("Reply from llm: ",completion.choices[0].message.content)
    except Exception as e:
        print("LLM error: ",e)
    reply=completion.choices[0].message.content
    if reply.startswith("```json"):
            reply = reply[7:-3].strip()
    elif reply.startswith("```"):
            reply = reply[3:-3].strip()
    structured_data=json.loads(reply)
    print(structured_data)
    return structured_data

In [ ]:
df = pd.DataFrame(columns=["Title", "Content"])
json_datas=[]
fileerror=[]
for file in files:
    with open(os.path.join(folder, file), "r") as f:
        try:
            json_data = json.load(f)
            
            if(len(json_data["Content"].split())>250):
                print("file: ",file)
                datas=chunksplit(json_data)
                for data in datas:
                    jsonofdata=data
                    json_datas.append(jsonofdata)
                    print("appended data:", jsonofdata["Title"])
            else:
                json_datas.append(json_data)
            print("Successfully appended file to datatset:",file)
        except Exception as e:
            print("file error: ",file)
            fileerror.append(file)
            continue

In [1]:
df = pd.DataFrame(json_datas)
df.to_csv("dataset.csv", index=True)
df.head()

NameError: name 'pd' is not defined

In [2]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df.head()
#sort df by title column alphabetically
df = df.sort_values(by="Title").reset_index(drop=True)
df.describe()

,Title,Content
count,221,221
unique,211,221
top,About Kerala Police,"At present, an ATM Counter of State Bank of In..."
freq,3,1


In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
import os

# ✅ Use valid NVIDIA embedding model
embeddings = NVIDIAEmbeddings(
    model="nvidia/llama-3.2-nemoretriever-300m-embed-v2",      # ← Correct model name
    api_key="",  # ← Proper env var
    truncate="END"  # or "NONE"; "END" is safer for long texts
)

# Build documents
docs = []
for index, row in df.iterrows():
    doc = Document(
        page_content=row['Content'],
        metadata={"title": row["Title"], "content": row["Content"]}
    )
    docs.append(doc)

print(f"Total documents prepared: {len(docs)}")

# Create & persist Chroma DB
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="vector_store",
    persist_directory="vector_store"
)
print("✅ Vector store created with NVIDIA embeddings and persisted.")

/home/mohammed.anas@acsiatech.com/Projects/Tasks/RAG_PROJECT/myenv/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_common.py:234: UserWarning: Found nvidia/llama-3.2-nemoretriever-300m-embed-v2 in available_models, but type is unknown and inference may fail.
  warnings.warn(


Total documents prepared: 221
✅ Vector store created with NVIDIA embeddings and persisted.


In [5]:
while True:
    query = input("Enter a query: ")

    print("→ Similarity:")
    for r in vectorstore.similarity_search(query, k=5):
       print(r.metadata)

→ Similarity:
{'title': 'About Kerala Police', 'content': "Kerala State Police is the law enforcement agency for the State of Kerala with its Headquarters at Thiruvananthapuram, the State Capital. Kerala Police has a reputation for being one of the best-managed state police forces in the country and is one of the top-ranking states in terms of maintenance of law and order. Kerala Police is also one of the first Police departments in South Asia to implement community policing through an enactment. It is popularly known as 'Janamaithri' Policing, which literally translates to people-friendly Policing. Kerala Police registers the highest number of cognizable offences in the country, next only to the National Capital Territory of Delhi. This has to be viewed in perspective of the relatively peaceful image of the state. This is a reflection of the good practice adopted by the state in registering every cognizable instance reported at the Police station level and dealing with the crimes and 

Exception: [###] Your request cannot be validated, it is incorrect. detail=Input list must be non-empty and all elements must be non-empty.
{'error': 'Your request cannot be validated, it is incorrect. detail=Input list must be non-empty and all elements must be non-empty.'}